In [1]:
import torchvision
import torch

In [2]:
from utils.dataloader import cifar_10_dataloader, view_sample_images
train_dataset, test_dataset, train_loader, test_loader = cifar_10_dataloader()

 
the class are   airplane--automobile--bird--cat--deer--dog--frog--horse--ship--truck

train_dataset.data.shape=(50000, 32, 32, 3)
test_dataset.data.shape=(10000, 32, 32, 3)


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

torchvision.transforms.transforms.ToTensor

In [4]:
# view_sample_images(train_dataset)

In [5]:
print(f"current accelerator is {torch.accelerator.current_accelerator()}")

current accelerator is mps


In [ ]:
from models.cifar10_simple_model import simple_model
from torchinfo import summary

model = simple_model().to(device)

summary(model, input_data=torch.randn(1,3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
simple_model                             [1, 10]                   --
├─Sequential: 1-1                        [1, 16, 32, 32]           --
│    └─Conv2d: 2-1                       [1, 16, 32, 32]           432
│    └─BatchNorm2d: 2-2                  [1, 16, 32, 32]           32
│    └─ReLU: 2-3                         [1, 16, 32, 32]           --
├─Sequential: 1-2                        [1, 16, 32, 32]           --
│    └─Conv2d: 2-4                       [1, 16, 32, 32]           2,304
│    └─BatchNorm2d: 2-5                  [1, 16, 32, 32]           32
│    └─ReLU: 2-6                         [1, 16, 32, 32]           --
├─MaxPool2d: 1-3                         [1, 16, 16, 16]           --
├─Sequential: 1-4                        [1, 32, 16, 16]           --
│    └─Conv2d: 2-7                       [1, 32, 16, 16]           4,608
│    └─BatchNorm2d: 2-8                  [1, 32, 16, 16]           64
│    └─R

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from tqdm import tqdm
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters())
epochs = 4
for epoch in range(epochs):
    correct,total = 0, 0
    model.train()
    train_pbar = tqdm(train_loader,desc=f"epoch = {epoch+1}")
    for images, labels in train_pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        _, prediction = torch.max(output,1)
        loss= criterion(output, labels)
        loss.backward()
        optimizer.step()
        correct+= sum(prediction==labels).item()
        total+= len(labels)
        train_pbar.set_postfix({
            'train_loss': loss.item(),
            'train_accuracy': correct/total
        })

    correct, total = 0, 0 
    model.eval()
    test_pbar = tqdm(test_loader,desc=f"epoch = {epoch+1}")
    with torch.no_grad():
        for images, labels in test_pbar:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, prediction = torch.max(output,1)
            loss= criterion(output, labels)
            correct+= sum(prediction==labels)
            total+= len(labels)
            test_pbar.set_postfix({
            'test_accuracy': correct/total
        })



epoch = 4:  49%|████▉     | 1542/3125 [00:30<00:26, 59.44it/s, train_loss=1.48, train_accuracy=0.485] 